In [1]:
import pandas as pd
import anndata

import scanpy as sc

In [2]:
# Read anndata
adata = sc.read('../data/adata_acini.h5ad')

In [3]:
# Create columns for spatial bins 
for ncell in range(4, 21, 4):
    new_colnm = 'mask_ncell{}'.format(ncell)
    adata.obs[new_colnm] = '-1'

In [4]:
# Transfer spatial bins mask into anndata
for sid in adata.obs.Batch.unique():
    fn = '../results/dfs_newlyassigned_rescaled2x/adata_cols_expand_{}.csv'.format(sid)
    if os.path.isfile(fn):
        tmp = pd.read_csv(fn, index_col=0)
        tmp = tmp.astype('str')
        for cnm in tmp.columns:
            adata.obs.loc[tmp.index, cnm] = tmp[cnm]

In [ ]:
# Write anndata
adata.write('../data/adata_final.h5ad')

In [ ]:
### Consolidate estimated area per mask for each sample

In [8]:
df = []
for sid in adata.obs.Batch.unique():
    fn = '../results/mask_coords_area/lummask_area_{}.csv'.format(sid)
    if os.path.isfile(fn):
        tmp = pd.read_csv(fn, index_col=0)
        tmp = tmp.transpose()
        if len(df) == 0:
            df = tmp.copy()
        else:
            df = pd.concat([df, tmp])


In [9]:
df.columns = ['mask_luminal']

In [12]:
for d_ncell in range(2, 21, 2):
    new_colnm = 'mask_ncell{}'.format(d_ncell)
    tmpdf = []
    for sid in adata.obs.Batch.unique():
        fn = '../results/dfs_newlyassigned_rescaled2x/area_{}_{}.csv'.format(sid, new_colnm)
        if os.path.isfile(fn):
            tmp = pd.read_csv(fn, index_col=0)
            if len(tmpdf) == 0:
                tmpdf = 10*tmp.copy()
            else:
                tmpdf = pd.concat([tmpdf, 10*tmp])
    df_area2[new_colnm] = tmpdf[new_colnm]

In [13]:
df_area = df.copy()

In [14]:
df_area['sid'] = [a.split('s1835')[-1] for a in df_area.index]

In [ ]:
# save consolidated area
df_area.to_csv('../results/area_all.csv')